In [2]:
import os
import numpy as np
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from concurrent.futures import ThreadPoolExecutor, as_completed
from numpy.linalg import norm
from tqdm import tqdm

os.environ["OPENAI_API_KEY"] = ''
client = OpenAI()

# RAG (Retrieval-Augmented Generation)

En esta práctica implementaremos un sistema RAG (Retrieval-Augmented Generation).

La revolución en torno a los LLM se basan en que escalando suficientemente un modelo basado en transformers como GPT-3 se ha descubierto que, si bien los modelos de completion no habían sido entrenado para razonar, ni responder preguntas, podían ser utilizados para ello!

Estas capacidades se denominan capacidades emergentes, ya que emergen del entrenamiento y escalamiento de la arquitectura: no hay un cambio de arquitectura, ni de entrenamiento.

Estas capacidades son propias del modelo, o sea están codificado en sus pesos. Dentro de los pesos se ha demostrado que se codifica conocimiento. Es así que los modelos pueden responder preguntas de conocimiento general. No obstante, si se quieren responder preguntas sobre data que no haya visto el modelo durante el entrenamiento (ya sea porque la información no haya estado disponible, sea privada o muy reciente o porque haya cambiado) entonces no responderá correctamente.

#### Y si usamos fine-tunning ?

Si bien es posible re-entrenar el modelo no es algo que sirva para agregar nuevo conocimiento: el conocimiento que adquieren estos modelos suele ser conocimiento que ve en reiteradas ocasiones en su corpus de entrenamiento. Además, como los pesos codifican otras capacidades que van desde básicas como razonar y manejar el lenguaje de forma coherente hasta conocimiento específico, modificar los pesos puede actuar en perjuicio de estas capacidades, desaprendiendo lo aprendido (problema conocido como "castastrofic forgetting"). Por otro lado, entrenarlos en corpus enteros como los que han sido usados para su creación lleva clusters de millones de dólares. El fine-tunning se suele hacer con técnicas que evitan el problema del "castastrofic forgetting" (como LoRa) y no para agregar nuevo conocimiento sino para afectar la manera en la que actúan (con técnicas como DPO o PPO). Para profundizar más pueden ver el siguiente tutorial https://huggingface.co/blog/dpo-trl

## Volviendo sobre RAG

Antes de los modelos generativos, la manera de responder consultas del usuario (estilo "que día nación San Martin?") sobre un corpus de datos, se solía encarar como un problema de Information Retrival: la respuesta consistía en hallar los textos y los párrafos en dichos textos que incluyera la información que buscaba el usuario.

### Construyendo un dataset sintético

Como retrieval corpus (o proprietary knowledge base), usaremos los libros en Inglés de Harry Potter, con un cambio fundamental, los nombres de los personajes estarán modificados.

Los libros de harry potter están en `data/practica_rag/harry_potter_books`
Además se encuentra una trivia en `data/practica_rag/harry_potter_trivia`

En ambos directorios se han aplicados los scripts de `TP2/harrypotter/name_replacement.py` para generar los archivos con los nombres modificados (que se encuentran en subdirectorios `data/practica_rag/harry_potter_books/_modified/` y `data/practica_rag/harry_potter_trivia/_modified/`).

### Calculando los embeddings

Se ha ejecutado el script `TP2/harrypotter/generate_embeddings.py` para generar los embeddings que han sido guardados en `data/practica_rag/embeddings.tar.xz` (al ser muy pesados han sido subidos de forma comprimida.

### Ejecutando el RAG

El RAG simplemente consiste, como se dijo, en:
- Calcular los párrafos relevantes. Para ello calculamos el embedding de la consulta y buscamos lo párrafos con embeddings de mayor similitud.
- Incluirlos en el contexto del prompt e instruir al modelo para que responda teniendo en cuenta el contexto.


In [4]:
embeddings_dir = '../data/practica_rag/embeddings/'
embeddings = np.load(os.path.join(embeddings_dir, "embeddings.npy"))
texts = np.load(os.path.join(embeddings_dir, "texts.npy"))

In [5]:
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [6]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))

def search_embeddings(query, top_k=5):
    query_embedding = get_embedding(query)
    similarities = np.array([cosine_similarity(query_embedding, emb) for emb in embeddings])
    top_k_indices = similarities.argsort()[-top_k:][::1]
    results = [texts[i] for i in top_k_indices[::-1]]
    return results

In [7]:
# Function to query GPT with search results as context
def query_gpt_with_context(query, top_k=10):
    context_results = search_embeddings(query, top_k=top_k)
    context = "\n\n".join(context_results)
    prompt = f"Context:\n{context}\n\nQuestion: {query}\n\nAnswer:"
    print(f"This is the full prompt with the context of selected fragments:\n\n {prompt}")
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "Answer the question based on the provided context."},
                  {"role": "user", "content": prompt}]
    )
    answer = response.choices[0].message.content
    return answer

Reemplacemos ahora los nombres en una lista de preguntas de Harry Potter, y pongamos a prueba nuestro RAG!

In [8]:
query = "What is the name of Villalba's faithful companion and dog"
answer = query_gpt_with_context(query)
print("Answer:", answer)

This is the full prompt with the context of selected fragments:

 Context:
Villalba was standing outside his hut, one hand on the collar of his enormous black boarhound, Fang. There were several open wooden crates on the ground at his feet, and Fang was whimpering and straining at his collar, apparently keen to investigate the contents more closely. As they drew nearer, an odd rattling noise reached their ears, punctuated by what sounded like minor explosions.

Roberto, Diego, and Carolina had always known that Villalba had an unfortunate liking for large and monstrous creatures. During their first year at Hogwarts he had tried to raise a dragon in his little wooden house, and it would be a long time before they forgot the giant, three- headed dog he'd christened "Fluffy." And if, as a boy, Villalba had heard that a monster was hidden somewhere in the castle, Roberto was sure he'd have gone to any lengths for a glimpse of it. He'd probably thought

Villalba was sitting in his shirtslee

#### Ejercicio C.1: "Evaluando nuestro RAG" (opcional)

Utilizar el código provisto para responder la trivia provista de forma automática, de manera que se compute una métrica de accuracy. 

Tener en cuenta que como hay varias maneras de expresar una respuesta correcta, no es correcto utilizar la comparación por igualdad.

#### Ejercicio C.2: "Reportando y analizando errores" (opcional)

Reportar 5 preguntas donde funcione mal. Tratar de determinar por qué (el retrival no encontró párrafos relevantes o el modelo se confunde al generar la respuesta?).

#### Ejercicio C.3: "Optimizando nuestro RAG" (opcional)

Probar variantes como incrementar el contexto, modificar el prompt o samplear varias veces la query al LLM y combinarlas de alguna forma y reportar si mejoran los resultados.

#### Ejercicio C.4: "Diseñando nuestro propio RAG" (opcional)

Diseñar tu propio sistema de RAG para un dataset propio.
Diseñar un dataset de preguntas y respuestas para evaluarlo.
Probar la accuracy del modelo sin retrival y con retrival. Si la data es pública, el modelo sin retrival debería funcionar razonablemente bien.